# 🔬 ArXiv Research Paper RAG Pipeline

**Retrieval-Augmented Generation for AI/ML Research Papers**

This notebook walks through building a complete RAG system that can answer questions about academic research papers using semantic search and LLM-powered generation.

## Pipeline Overview

```
📄 PDFs → 📝 Text Extraction → ✂️ Chunking → 🧮 Embedding → 📦 FAISS Index
                                                                    ↓
                                    🤖 LLM Answer ← 📋 Context ← 🔍 Retrieval ← ❓ Query
```

| Component | Choice | Justification |
|-----------|--------|---------------|
| Chunking | 512 tokens, 50 overlap | Captures 1-2 academic paragraphs; overlap prevents boundary loss |
| Embedding | `all-MiniLM-L6-v2` (384d) | Best quality/speed ratio, 14 MB, trained on 1B+ pairs |
| Vector DB | FAISS `IndexFlatIP` | Open-source, no server, exact inner product search |
| LLM | Configurable (OpenAI/Ollama) | Flexible for cloud or local inference |

---
## Cell 1: Setup & Imports

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 1: Setup & Imports
# ═══════════════════════════════════════════════════════════

import sys
import json
import logging
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to path
PROJECT_ROOT = Path(".").resolve().parent
sys.path.insert(0, str(PROJECT_ROOT / "src"))

# Import our pipeline components
from config import (
    PAPERS_DIR, METADATA_FILE, FAISS_INDEX_DIR,
    CHUNK_SIZE, CHUNK_OVERLAP, EMBEDDING_MODEL_NAME, TOP_K,
)
from rag_pipeline import (
    PDFTextExtractor, TextChunker, EmbeddingEngine,
    VectorStore, RAGPipeline,
)

# Configure display
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")
pd.set_option("display.max_colwidth", 100)
plt.style.use("seaborn-v0_8-whitegrid")
sns.set_palette("husl")

print(f"✅ Project root: {PROJECT_ROOT}")
print(f"✅ Papers dir:   {PAPERS_DIR}")
print(f"✅ Chunk size:   {CHUNK_SIZE} chars, {CHUNK_OVERLAP} overlap")
print(f"✅ Embed model:  {EMBEDDING_MODEL_NAME}")

---
## Cell 2: Data Loading & Exploration

Load the metadata JSON created by `collect_papers.py` and explore the dataset.

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 2: Data Loading & Exploration
# ═══════════════════════════════════════════════════════════

# Load metadata
with open(METADATA_FILE, "r", encoding="utf-8") as f:
    metadata = json.load(f)

df = pd.DataFrame(metadata)
print(f"📊 Dataset Summary")
print(f"{'='*50}")
print(f"Total papers:       {len(df)}")
print(f"Unique categories:  {df['primary_category'].nunique()}")
print(f"Date range:         {df['published'].min()[:10]} to {df['published'].max()[:10]}")
print(f"\n📁 Category Distribution:")
print(df['primary_category'].value_counts().to_string())

# Show sample papers
print(f"\n📄 Sample Papers:")
df[['title', 'primary_category', 'published']].head(10)

---
## Cell 3: PDF Text Extraction

Extract text from all downloaded PDFs using `pdfplumber` (primary) with `PyPDF2` fallback.

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 3: PDF Text Extraction
# ═══════════════════════════════════════════════════════════

extractor = PDFTextExtractor()
documents = extractor.extract_all(PAPERS_DIR, METADATA_FILE)

print(f"\n📄 Text Extraction Results")
print(f"{'='*50}")
print(f"Successfully extracted: {len(documents)}/{len(metadata)} papers")

# Text length statistics
text_lengths = [len(doc['text']) for doc in documents]
print(f"\n📏 Text Length Statistics (characters):")
print(f"  Mean:   {np.mean(text_lengths):,.0f}")
print(f"  Median: {np.median(text_lengths):,.0f}")
print(f"  Min:    {np.min(text_lengths):,.0f}")
print(f"  Max:    {np.max(text_lengths):,.0f}")
print(f"  Total:  {sum(text_lengths):,.0f}")

# Show a sample extracted text
sample = documents[0]
print(f"\n📝 Sample — '{sample['title'][:60]}...'")
print(f"   First 500 chars:")
print(f"   {sample['text'][:500]}...")

---
## Cell 4: Chunking Strategy

Split documents into overlapping chunks using `RecursiveCharacterTextSplitter`.

**Why 512 tokens, 50 overlap?**
- Academic paragraphs average 150–300 words (~200–400 tokens)
- 512 tokens captures 1–2 complete paragraphs — enough context without dilution
- 50-token overlap (~10%) prevents boundary information loss

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 4: Chunking Strategy
# ═══════════════════════════════════════════════════════════

chunker = TextChunker(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
)
chunks = chunker.chunk_documents(documents)

# Chunking statistics
stats = chunker.get_stats(chunks)
print(f"\n✂️  Chunking Results")
print(f"{'='*50}")
for key, value in stats.items():
    if isinstance(value, float):
        print(f"  {key:25s}: {value:.1f}")
    else:
        print(f"  {key:25s}: {value}")

# Visualize chunk length distribution
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

chunk_lengths = [len(c['text']) for c in chunks]

axes[0].hist(chunk_lengths, bins=50, color='#4C72B0', edgecolor='white', alpha=0.8)
axes[0].axvline(x=CHUNK_SIZE, color='red', linestyle='--', label=f'Target: {CHUNK_SIZE}')
axes[0].set_xlabel('Chunk Length (characters)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Chunk Length Distribution')
axes[0].legend()

# Chunks per paper
chunks_per_paper = {}
for c in chunks:
    chunks_per_paper[c['paper_id']] = chunks_per_paper.get(c['paper_id'], 0) + 1
cpv = list(chunks_per_paper.values())

axes[1].hist(cpv, bins=30, color='#55A868', edgecolor='white', alpha=0.8)
axes[1].set_xlabel('Number of Chunks')
axes[1].set_ylabel('Number of Papers')
axes[1].set_title('Chunks per Paper Distribution')

plt.tight_layout()
plt.savefig(str(PROJECT_ROOT / 'data' / 'chunk_distribution.png'), dpi=150, bbox_inches='tight')
plt.show()

# Show sample chunks from one paper
sample_id = chunks[0]['paper_id']
sample_chunks = [c for c in chunks if c['paper_id'] == sample_id]
print(f"\n📋 Sample chunks from '{sample_chunks[0]['paper_title'][:50]}...':")
for c in sample_chunks[:3]:
    print(f"\n  Chunk {c['chunk_index']+1}/{c['total_chunks']} ({len(c['text'])} chars):")
    print(f"  {c['text'][:150]}...")

---
## Cell 5: Embedding Generation

Generate dense vector embeddings using `all-MiniLM-L6-v2`.

**Model Details:**
- 384 dimensions, ~22M parameters, ~14 MB
- Trained on 1B+ sentence pairs (NLI + semantic similarity)
- Maps text to a 384-dimensional dense vector space
- L2-normalized → inner product = cosine similarity

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 5: Embedding Generation
# ═══════════════════════════════════════════════════════════

import time

engine = EmbeddingEngine(model_name=EMBEDDING_MODEL_NAME)

# Embed all chunks
texts = [c['text'] for c in chunks]

start_time = time.time()
embeddings = engine.embed_texts(texts, batch_size=64)
elapsed = time.time() - start_time

print(f"\n🧮 Embedding Results")
print(f"{'='*50}")
print(f"  Model:         {EMBEDDING_MODEL_NAME}")
print(f"  Chunks:        {embeddings.shape[0]}")
print(f"  Dimensions:    {embeddings.shape[1]}")
print(f"  Time:          {elapsed:.1f}s ({len(texts)/elapsed:.0f} chunks/sec)")
print(f"  Memory:        {embeddings.nbytes / 1024 / 1024:.1f} MB")
print(f"  Dtype:         {embeddings.dtype}")

# Verify L2 normalization (norms should be ~1.0)
norms = np.linalg.norm(embeddings, axis=1)
print(f"\n  L2 norms — mean: {norms.mean():.4f}, std: {norms.std():.6f}")

# Show a sample embedding vector
print(f"\n  Sample embedding (first 10 dims):")
print(f"  {embeddings[0][:10]}")

---
## Cell 6: FAISS Vector Store Creation

Build a FAISS index for fast similarity search.

**Why FAISS?**
- Open-source (Meta AI), no external server needed
- Extremely fast approximate nearest neighbor search
- `IndexFlatIP` — exact inner product search (cosine sim for normalized vectors)
- Perfectly scaled for our dataset (5K–50K chunks)

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 6: FAISS Index Creation
# ═══════════════════════════════════════════════════════════

store = VectorStore(dimension=engine.dimension)
store.build_index(embeddings, chunks)

# Save to disk
store.save(FAISS_INDEX_DIR)

# Print index statistics
index_stats = store.get_stats()
print(f"\n📦 FAISS Index Statistics")
print(f"{'='*50}")
for key, value in index_stats.items():
    print(f"  {key:20s}: {value}")

print(f"\n  Index type:    {type(store.index).__name__}")
print(f"  Saved to:      {FAISS_INDEX_DIR}")

---
## Cell 7: Query Engine — Semantic Search

Test the retrieval component independently before wiring up the LLM.

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 7: Query Engine — Semantic Search
# ═══════════════════════════════════════════════════════════

def semantic_search(query: str, top_k: int = TOP_K) -> list[dict]:
    """Search the FAISS index for the most relevant chunks."""
    query_embedding = engine.embed_query(query)
    results = store.search(query_embedding, top_k=top_k)
    return results


def display_results(query: str, results: list[dict]):
    """Pretty-print search results."""
    print(f"\n🔍 Query: \"{query}\"")
    print(f"{'─'*60}")
    for i, r in enumerate(results, 1):
        print(f"\n  [{i}] Score: {r['score']:.4f}")
        print(f"      Paper: {r['paper_title'][:70]}")
        print(f"      Chunk: {r['chunk_index']+1}/{r['total_chunks']}")
        print(f"      Text:  {r['text'][:200]}...")
    print(f"{'─'*60}")


# Test queries
test_queries = [
    "What are transformer architectures and how do they work?",
    "How does attention mechanism improve neural networks?",
    "What are the latest advances in computer vision?",
]

for q in test_queries:
    results = semantic_search(q, top_k=3)
    display_results(q, results)

---
## Cell 8: RAG Chain with LLM

Wire the retrieval engine into a full RAG chain with LLM-powered generation.

> **Note:** Set `OPENAI_API_KEY` in your `.env` file, or start Ollama locally. If no LLM is configured, the system will return raw retrieved chunks.

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 8: RAG Chain with LLM
# ═══════════════════════════════════════════════════════════

from dotenv import load_dotenv
load_dotenv()  # Load OPENAI_API_KEY from .env if present

# Create the LLM
llm = RAGPipeline.create_llm()

# Build the full RAG pipeline
rag = RAGPipeline(
    embedding_engine=engine,
    vector_store=store,
    llm=llm,
    top_k=TOP_K,
)

print(f"✅ RAG Pipeline initialized")
print(f"   LLM: {'Configured ✓' if llm else 'None (will return raw context) ⚠️'}")
print(f"   Top-K: {TOP_K}")

# Test the pipeline
result = rag.query("What are the key innovations in transformer architectures?")

print(f"\n{'='*60}")
print(f"❓ Question: {result['question']}")
print(f"{'='*60}")
print(f"\n💬 Answer:\n{result['answer'][:1000]}")
print(f"\n📚 Sources ({result['num_sources']}):")
for s in result['sources']:
    print(f"   [{s['score']:.4f}] {s['paper_title'][:60]}")

---
## Cell 9: Test Queries

Run a diverse set of research questions to evaluate the RAG system's capabilities.

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 9: Test Queries
# ═══════════════════════════════════════════════════════════

test_questions = [
    # Architecture questions
    "What are the main types of neural network architectures used in NLP?",
    
    # Comparison questions
    "How do CNNs compare to transformers for image classification tasks?",
    
    # Methodology questions
    "What training techniques are used to improve model generalization?",
    
    # Application questions
    "What are the real-world applications of reinforcement learning?",
    
    # Recent advances
    "What are the latest developments in large language models?",
]

all_results = []

for i, question in enumerate(test_questions, 1):
    print(f"\n{'═'*70}")
    print(f"  Query {i}: {question}")
    print(f"{'═'*70}")
    
    result = rag.query(question, top_k=5)
    all_results.append(result)
    
    # Show answer (truncated)
    answer_preview = result['answer'][:500]
    print(f"\n💬 Answer:\n{answer_preview}{'...' if len(result['answer']) > 500 else ''}")
    
    # Show sources
    print(f"\n📚 Top Sources:")
    for s in result['sources'][:3]:
        print(f"   [{s['score']:.4f}] {s['paper_title'][:60]}")

print(f"\n\n✅ Completed {len(test_questions)} test queries")

---
## Cell 10: Evaluation & Metrics

Evaluate the retrieval quality with quantitative metrics.

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 10: Evaluation & Metrics
# ═══════════════════════════════════════════════════════════

import time

eval_queries = [
    "transformer architecture self-attention mechanism",
    "convolutional neural networks image recognition",
    "natural language processing text generation",
    "reinforcement learning policy optimization",
    "generative adversarial networks image synthesis",
    "transfer learning domain adaptation",
    "graph neural networks node classification",
    "federated learning privacy preservation",
]

# Metric 1: Retrieval Latency
latencies = []
for q in eval_queries:
    start = time.time()
    _ = semantic_search(q, top_k=5)
    latencies.append((time.time() - start) * 1000)  # ms

# Metric 2: Score Distribution
all_scores = []
all_top1_scores = []
for q in eval_queries:
    results = semantic_search(q, top_k=10)
    scores = [r['score'] for r in results]
    all_scores.extend(scores)
    all_top1_scores.append(scores[0] if scores else 0)

# Metric 3: Source Diversity
diversity_scores = []
for q in eval_queries:
    results = semantic_search(q, top_k=5)
    unique_papers = len(set(r['paper_id'] for r in results))
    diversity_scores.append(unique_papers / max(len(results), 1))

# Print metrics
print(f"📊 Retrieval Evaluation Metrics")
print(f"{'='*50}")
print(f"\n⏱️  Latency (ms):")
print(f"   Mean:   {np.mean(latencies):.1f}")
print(f"   Median: {np.median(latencies):.1f}")
print(f"   P95:    {np.percentile(latencies, 95):.1f}")
print(f"   Max:    {np.max(latencies):.1f}")

print(f"\n🎯 Similarity Scores (cosine):")
print(f"   Top-1 avg:    {np.mean(all_top1_scores):.4f}")
print(f"   Overall avg:  {np.mean(all_scores):.4f}")
print(f"   Overall std:  {np.std(all_scores):.4f}")
print(f"   Score range:  [{np.min(all_scores):.4f}, {np.max(all_scores):.4f}]")

print(f"\n🌐 Source Diversity (unique papers / top-k):")
print(f"   Mean:   {np.mean(diversity_scores):.2f}")
print(f"   Min:    {np.min(diversity_scores):.2f}")

# Visualization
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

axes[0].bar(range(len(latencies)), latencies, color='#4C72B0', alpha=0.8)
axes[0].axhline(y=np.mean(latencies), color='red', linestyle='--', label=f'Mean: {np.mean(latencies):.1f}ms')
axes[0].set_xlabel('Query Index')
axes[0].set_ylabel('Latency (ms)')
axes[0].set_title('Retrieval Latency per Query')
axes[0].legend()

axes[1].hist(all_scores, bins=30, color='#55A868', edgecolor='white', alpha=0.8)
axes[1].set_xlabel('Cosine Similarity Score')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Score Distribution (All Retrieved)')

axes[2].bar(range(len(diversity_scores)), diversity_scores, color='#C44E52', alpha=0.8)
axes[2].set_xlabel('Query Index')
axes[2].set_ylabel('Diversity Score')
axes[2].set_title('Source Diversity per Query')
axes[2].set_ylim(0, 1.1)

plt.tight_layout()
plt.savefig(str(PROJECT_ROOT / 'data' / 'evaluation_metrics.png'), dpi=150, bbox_inches='tight')
plt.show()

---
## Cell 11: Visualization & Analysis

Visualize the embedding space using t-SNE dimensionality reduction.

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 11: Visualization & Analysis
# ═══════════════════════════════════════════════════════════

from sklearn.manifold import TSNE

# Sample embeddings for visualization (use all if < 2000, else sample)
max_vis = 2000
if len(embeddings) > max_vis:
    indices = np.random.choice(len(embeddings), max_vis, replace=False)
    vis_embeddings = embeddings[indices]
    vis_chunks = [chunks[i] for i in indices]
else:
    vis_embeddings = embeddings
    vis_chunks = chunks

# t-SNE reduction to 2D
print("🔄 Running t-SNE (this may take a minute)...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
coords_2d = tsne.fit_transform(vis_embeddings)

# Color by primary category
categories = [c.get('categories', ['unknown'])[0] if c.get('categories') else 'unknown' for c in vis_chunks]
unique_cats = list(set(categories))
cat_to_color = {cat: i for i, cat in enumerate(unique_cats)}
colors = [cat_to_color[cat] for cat in categories]

# Plot
fig, ax = plt.subplots(1, 1, figsize=(12, 10))

scatter = ax.scatter(
    coords_2d[:, 0], coords_2d[:, 1],
    c=colors, cmap='tab20', alpha=0.6, s=15,
    edgecolors='none',
)

# Add legend
handles = []
for cat in unique_cats[:10]:  # Show top 10 categories
    color = plt.cm.tab20(cat_to_color[cat] / len(unique_cats))
    handles.append(plt.scatter([], [], c=[color], label=cat, s=50))
ax.legend(handles=handles, title='Category', loc='upper right', fontsize=8)

ax.set_title('t-SNE Visualization of Paper Chunk Embeddings', fontsize=14)
ax.set_xlabel('t-SNE Dimension 1')
ax.set_ylabel('t-SNE Dimension 2')

plt.tight_layout()
plt.savefig(str(PROJECT_ROOT / 'data' / 'embedding_tsne.png'), dpi=150, bbox_inches='tight')
plt.show()

print(f"\n✅ t-SNE plot saved to data/embedding_tsne.png")
print(f"   Visualized {len(vis_embeddings)} chunks across {len(unique_cats)} categories")

# Retrieval heatmap for test queries
print("\n📊 Generating retrieval score heatmap...")

heatmap_queries = [
    "transformer architecture",
    "attention mechanism",
    "image classification CNN",
    "language model pretraining",
    "reinforcement learning",
]

heatmap_data = []
for q in heatmap_queries:
    results = semantic_search(q, top_k=10)
    scores = [r['score'] for r in results]
    heatmap_data.append(scores)

fig, ax = plt.subplots(figsize=(12, 5))
sns.heatmap(
    heatmap_data,
    xticklabels=[f'Rank {i+1}' for i in range(10)],
    yticklabels=[q[:30] for q in heatmap_queries],
    annot=True, fmt='.3f', cmap='YlOrRd',
    ax=ax,
)
ax.set_title('Retrieval Similarity Scores Heatmap')
ax.set_xlabel('Retrieval Rank')

plt.tight_layout()
plt.savefig(str(PROJECT_ROOT / 'data' / 'retrieval_heatmap.png'), dpi=150, bbox_inches='tight')
plt.show()
print("✅ Heatmap saved to data/retrieval_heatmap.png")

---
## Cell 12: Summary & Future Work

Key takeaways, limitations, and potential improvements.

In [ ]:
# ═══════════════════════════════════════════════════════════
#  Cell 12: Summary & Future Work
# ═══════════════════════════════════════════════════════════

# Pipeline summary
summary = {
    "Papers Processed": len(documents),
    "Total Chunks": len(chunks),
    "Embedding Model": EMBEDDING_MODEL_NAME,
    "Embedding Dimension": engine.dimension,
    "Chunk Size": CHUNK_SIZE,
    "Chunk Overlap": CHUNK_OVERLAP,
    "FAISS Index Vectors": store.index.ntotal,
    "Avg Retrieval Latency": f"{np.mean(latencies):.1f}ms",
    "Avg Top-1 Score": f"{np.mean(all_top1_scores):.4f}",
}

print("\n" + "═"*60)
print("  📋 RAG PIPELINE SUMMARY")
print("═"*60)
for key, value in summary.items():
    print(f"  {key:30s}: {value}")

print(f"\n\n{'═'*60}")
print("  🔮 FUTURE IMPROVEMENTS")
print(f"{'═'*60}")

improvements = [
    ("Hybrid Search", "Combine dense (semantic) + sparse (BM25) retrieval for better precision"),
    ("Re-ranking", "Add a cross-encoder re-ranker (e.g., ms-marco-MiniLM) on top-k results"),
    ("Metadata Filtering", "Filter by category, date, or author before/after retrieval"),
    ("Multi-Query", "Generate multiple query variations for broader recall"),
    ("Chunk Deduplication", "Remove near-duplicate chunks using MinHash or cosine threshold"),
    ("Evaluation Framework", "Use RAGAS or DeepEval for automated RAG evaluation metrics"),
    ("Streaming Generation", "Stream LLM responses for better UX in the Streamlit app"),
    ("Citation Linking", "Link answers back to specific pages/sections in the source PDFs"),
]

for title, desc in improvements:
    print(f"\n  🔹 {title}")
    print(f"     {desc}")

print(f"\n\n{'═'*60}")
print("  ✅ Pipeline complete! Run the Streamlit app for an interactive UI:")
print("     streamlit run app/streamlit_app.py")
print(f"{'═'*60}")